# 投资者情绪专题
使用所有的数据进行加总

## 数据准备

In [ ]:
import sys
import os
import cudf  #CUDA计算
import pandas as pd

# 自行编写的包
sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')

# Stata
from pystata import config  # noqa

config.init('mp')
# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = '/data/DataSets/investor_sentiment/'

合并Excel

In [ ]:
def excel_transform():
    """
    转换excel
    :return:
    """

    import multiprocessing as mp

    file_path = '/home/ubuntu/notebook/DataSets/FORUM_SENT/'
    pool = mp.Pool(10)

    # 任务函数
    def myfunc(name):
        df = pd.read_excel(file_path + name, header=0, engine='openpyxl', skiprows=[1, 2], dtype={'Stockcode': 'str', 'PostDate': 'str'})
        out_name = file_path + name[:-5] + '.parquet'
        df.to_parquet(out_name)
        return {name: out_name}

    results = [pool.apply_async(myfunc, args=(name,)) for name in os.listdir(file_path) if '.xlsx' in name]
    results = [p.get() for p in results]
    return results


def merge_parquet():
    df_merge = pd.DataFrame()
    for i in os.listdir(file_path):
        if '.parquet' in i:
            df_merge = pd.concat([df_merge, pd.read_parquet(file_path + i)])
    df_merge.to_parquet('/data/DataSets/investor_sentiment/FORUM_SENT.parquet')